In [1]:
#!pip install gym
#import libraries 
import gym
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np
import time, math, random
from typing import Tuple

In [2]:
#CartPole-v1 Environment
env = gym.make('CartPole-v1')

In [3]:
#!pip install pyglet
#Visualise the eniroment or simulation

env_policy = lambda obs: 1
for _ in range(5):
    obs = env.reset()
    for _ in range(80):
        actions = env_policy(obs)
        obs, reward, done, info = env.step(actions) 
        env.render()
        time.sleep(0.05)

env.close()

/Users/yousef/opt/anaconda3/lib/python3.8/site-packages/gym/envs/classic_control/cartpole.py:150: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  logger.warn(


In [4]:
#Look at the docstring
?env.env

In [5]:
#Simple policy function 
env_policy = lambda _,__,___, tip_velocity : int( tip_velocity > 0 )


In [6]:
#This is to to convert Catpoles continues state space into discrete one
n_buckets = ( 6 , 12 )
lower_bounds = [ env.observation_space.low[2], -math.radians(50) ]
upper_bounds = [ env.observation_space.high[2], math.radians(50) ]

def discretizer_range( _ , __ , angle, pole_velocity ) -> Tuple[int,...]:
    """Convert continues state intro a discrete state"""
    estimator = KBinsDiscretizer(n_bins=n_buckets, encode='ordinal', strategy='uniform')
    estimator.fit([lower_bounds, upper_bounds ])
    return tuple(map(int,estimator.transform([[angle, pole_velocity]])[0]))

In [8]:
#Initialize the Q value table 
Q_table = np.zeros(n_buckets + (env.action_space.n,))
Q_table.shape

(6, 12, 2)

In [9]:
#Create a polict function and take the max Q value 
def env_policy( state : tuple ):
    """Choosing action based on epsilon-greedy policy"""
    return np.argmax(Q_table[state])

In [10]:
#update func
def new_Q_value( reward : float ,  new_state : tuple , discount_factor=1 ) -> float:
    """Temperal diffrence for updating Q-value of state-action pair"""
    future_optimal_value = np.max(Q_table[new_state])
    learned_value = reward + discount_factor * future_optimal_value
    return learned_value

In [11]:
# Adaptive learning of Learning Rate
def learning_rate(n : int , min_rate=0.01 ) -> float  :
    """Decaying learning rate"""
    return max(min_rate, min(1.0, 1.0 - math.log10((n + 1) / 25)))

In [12]:
def exploration_rate(n : int, min_rate= 0.1 ) -> float :
    """Decaying exploration rate"""
    return max(min_rate, min(1, 1.0 - math.log10((n  + 1) / 25)))

In [ ]:
# Train it
num_episodes = 10000
for e in range(num_episodes):
    
    # Siscretize state into buckets
    current_state, done = discretizer_range(*env.reset()), False
    
    while done==False:
        
        # policy action || exploit
        action = np.argmax(Q_table[current_state])
        
        # insert random action || explore 
        if np.random.random() < exploration_rate(e) : 
            action = env.action_space.sample() 
         
        # increment enviroment
        obs, reward, done, _ = env.step(action)
        new_state = discretizer_range(*obs)
        
        # Update Q-Table
        lr = learning_rate(e)
        learnt_value = new_Q_value(reward , new_state )
        old_value = Q_table[current_state][action]
        Q_table[current_state][action] = (1-lr)*old_value + lr*learnt_value
        
        current_state = new_state
        
        # Render the cartpole environment
        env.render()